In [ ]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=2562f25c78048a75d0f603b71ffd069bca1d6a27e82af703abb9cb01d79cb883
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Loading Package
import pyspark
from pyspark import SparkContext

In [ ]:
sc = SparkContext.getOrCreate();

In [ ]:
#creating spark session
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("BDdiabetes").getOrCreate()

In [ ]:
#Load Dataset
df = spark.read.csv('diabetes.csv',header=True, inferSchema=True)

In [ ]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


In [ ]:
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [ ]:
df.columns

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [ ]:
df.groupBy('Outcome').count().show()

+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  268|
|      0|  500|
+-------+-----+



In [ ]:
import pyspark.ml

In [ ]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [ ]:
#Label Encoding
LabelEncoding = StringIndexer(inputCol='Outcome', outputCol='Target').fit(df)
df = LabelEncoding.transform(df)

In [ ]:
#get the labels
LabelEncoding.labels

['0', '1']

In [ ]:
df2 = df.select('Outcome')

In [ ]:
df2.printSchema()

root
 |-- Outcome: integer (nullable = true)



In [ ]:
required_features = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']

In [ ]:
#vector assembler
vec_assembler = VectorAssembler(inputCols=required_features, outputCol='features')

In [ ]:
vec_df = vec_assembler.transform(df)

In [ ]:
vec_df.show(3)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|Target|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+------+--------------------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|   1.0|[6.0,148.0,72.0,3...|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|   0.0|[1.0,85.0,66.0,29...|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|   1.0|[8.0,183.0,64.0,0...|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+------+--------------------+
only showing top 3 rows



In [ ]:
finalased_data = vec_df.select('features','Target')

In [ ]:
finalased_data.show()

+--------------------+------+
|            features|Target|
+--------------------+------+
|[6.0,148.0,72.0,3...|   1.0|
|[1.0,85.0,66.0,29...|   0.0|
|[8.0,183.0,64.0,0...|   1.0|
|[1.0,89.0,66.0,23...|   0.0|
|[0.0,137.0,40.0,3...|   1.0|
|[5.0,116.0,74.0,0...|   0.0|
|[3.0,78.0,50.0,32...|   1.0|
|[10.0,115.0,0.0,0...|   0.0|
|[2.0,197.0,70.0,4...|   1.0|
|[8.0,125.0,96.0,0...|   1.0|
|[4.0,110.0,92.0,0...|   0.0|
|[10.0,168.0,74.0,...|   1.0|
|[10.0,139.0,80.0,...|   0.0|
|[1.0,189.0,60.0,2...|   1.0|
|[5.0,166.0,72.0,1...|   1.0|
|[7.0,100.0,0.0,0....|   1.0|
|[0.0,118.0,84.0,4...|   1.0|
|[7.0,107.0,74.0,0...|   1.0|
|[1.0,103.0,30.0,3...|   0.0|
|[1.0,115.0,70.0,3...|   1.0|
+--------------------+------+
only showing top 20 rows



In [ ]:
train_df,test_df = vec_df.randomSplit([0.7,0.3])

In [ ]:
train_df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|Target|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+------+--------------------+
|          0|     73|            0|            0|      0|21.1|                   0.342| 25|      0|   0.0|(8,[1,5,6,7],[73....|
|          0|     74|           52|           10|     36|27.8|                   0.269| 22|      0|   0.0|[0.0,74.0,52.0,10...|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|   0.0|[0.0,84.0,82.0,31...|
|          0|     86|           68|           32|      0|35.8|                   0.238| 25|      0|   0.0|[0.0,86.0,68.0,32...|
|          0|     91|           68|           32|    210|39.9|                   0.381| 25|      0|   0.

In [ ]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier

##Logistic Regression

In [ ]:
LR = LogisticRegression(featuresCol='features', labelCol='Target')

In [ ]:
LR_Model = LR.fit(train_df)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame({"Coefficients":LR_Model.coefficients}, index=required_features)

,Coefficients
Pregnancies,0.137684
Glucose,0.034615
BloodPressure,-0.009749
SkinThickness,0.000941
Insulin,-0.001419
BMI,0.066169
DiabetesPedigreeFunction,1.074135
Age,0.004124


In [ ]:
y_pred = LR_Model.transform(test_df)

In [ ]:
y_pred.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+------+--------------------+--------------------+--------------------+----------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|Target|            features|       rawPrediction|         probability|prediction|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+------+--------------------+--------------------+--------------------+----------+
|          0|     57|           60|            0|      0|21.7|                   0.735| 67|      0|   0.0|[0.0,57.0,60.0,0....|[3.58462386282940...|[0.97300201351108...|       0.0|
|          0|     67|           76|            0|      0|45.3|                   0.194| 46|      0|   0.0|[0.0,67.0,76.0,0....|[2.52501573197027...|[0.92587701448231...|       0.0|
|          0|     78|           88|           29|     40|36.9|                   0.434| 21|    

In [ ]:
print(y_pred.columns)

['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'Target', 'features', 'rawPrediction', 'probability', 'prediction']


In [ ]:
y_pred.select('target','rawPrediction','probability','prediction').show()

+------+--------------------+--------------------+----------+
|target|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+----------+
|   0.0|[3.35765837024926...|[0.96635472576344...|       0.0|
|   0.0|[2.33809395357971...|[0.91198320723038...|       0.0|
|   0.0|[2.75692312764986...|[0.94030315334357...|       0.0|
|   0.0|[2.51803653302550...|[0.92539661472056...|       0.0|
|   0.0|[1.29812968061042...|[0.78552004307604...|       0.0|
|   0.0|[1.78143841627449...|[0.85587439031724...|       0.0|
|   0.0|[3.02259787178574...|[0.95358464539638...|       0.0|
|   0.0|[2.14610962429876...|[0.89530467597985...|       0.0|
|   0.0|[0.82790749508525...|[0.69591229895912...|       0.0|
|   0.0|[2.72786199603283...|[0.93865083467290...|       0.0|
|   0.0|[2.73468576928481...|[0.93904261048915...|       0.0|
|   0.0|[4.06748048450509...|[0.98316770725333...|       0.0|
|   0.0|[2.27203829372644...|[0.90653463472311...|       0.0|
|   0.0|

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
multi_evaluator = MulticlassClassificationEvaluator(labelCol='Target',metricName='accuracy')

In [ ]:
multi_evaluator.evaluate(y_pred)

0.8076923076923077

##DecisionTreeClassifier

In [ ]:
DT = DecisionTreeClassifier(featuresCol='features', labelCol='Target')

In [ ]:
DT_Model = DT.fit(train_df)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame({"Coefficients":LR_Model.coefficients}, index=required_features)

,Coefficients
Pregnancies,0.137684
Glucose,0.034615
BloodPressure,-0.009749
SkinThickness,0.000941
Insulin,-0.001419
BMI,0.066169
DiabetesPedigreeFunction,1.074135
Age,0.004124


In [ ]:
DT_pred = DT_Model.transform(test_df)

In [ ]:
DT_pred.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+------+--------------------+-------------+--------------------+----------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|Target|            features|rawPrediction|         probability|prediction|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+------+--------------------+-------------+--------------------+----------+
|          0|     74|           52|           10|     36|27.8|                   0.269| 22|      0|   0.0|[0.0,74.0,52.0,10...|  [104.0,1.0]|[0.99047619047619...|       0.0|
|          0|     84|           64|           22|     66|35.8|                   0.545| 21|      0|   0.0|[0.0,84.0,64.0,22...|   [46.0,5.0]|[0.90196078431372...|       0.0|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|   0.0|[0.0,84.0,82.0,31...|   

In [ ]:
print(DT_pred.columns)

['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'Target', 'features', 'rawPrediction', 'probability', 'prediction']


In [ ]:
DT_pred.select('target','rawPrediction','probability','prediction').show()

+------+-------------+--------------------+----------+
|target|rawPrediction|         probability|prediction|
+------+-------------+--------------------+----------+
|   0.0|  [104.0,1.0]|[0.99047619047619...|       0.0|
|   0.0|   [46.0,5.0]|[0.90196078431372...|       0.0|
|   0.0|   [46.0,5.0]|[0.90196078431372...|       0.0|
|   0.0|   [46.0,5.0]|[0.90196078431372...|       0.0|
|   0.0|    [9.0,5.0]|[0.64285714285714...|       0.0|
|   0.0|   [46.0,5.0]|[0.90196078431372...|       0.0|
|   0.0|  [104.0,1.0]|[0.99047619047619...|       0.0|
|   0.0|  [104.0,1.0]|[0.99047619047619...|       0.0|
|   0.0|    [9.0,5.0]|[0.64285714285714...|       0.0|
|   0.0|  [104.0,1.0]|[0.99047619047619...|       0.0|
|   0.0|  [104.0,1.0]|[0.99047619047619...|       0.0|
|   0.0|  [104.0,1.0]|[0.99047619047619...|       0.0|
|   0.0|   [46.0,5.0]|[0.90196078431372...|       0.0|
|   0.0|   [26.0,6.0]|     [0.8125,0.1875]|       0.0|
|   1.0|    [2.0,5.0]|[0.28571428571428...|       1.0|
|   0.0|  

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
multi_evaluator = MulticlassClassificationEvaluator(labelCol='Target',metricName='accuracy')

In [ ]:
multi_evaluator.evaluate(DT_pred)

0.7393162393162394

##RandomForestClassifier

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
RF = RandomForestClassifier(featuresCol='features', labelCol='Target')

In [ ]:
RF_Model = RF.fit(train_df)

In [ ]:
RF_pred = RF_Model.transform(test_df)

In [ ]:
RF_pred.show()

In [ ]:
RF_pred.select('target','rawPrediction','probability','prediction').show()

+------+--------------------+--------------------+----------+
|target|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+----------+
|   0.0|[19.0510579915350...|[0.95255289957675...|       0.0|
|   0.0|[17.4989962388558...|[0.87494981194279...|       0.0|
|   0.0|[16.4095559101595...|[0.82047779550797...|       0.0|
|   0.0|[17.4428405355814...|[0.87214202677907...|       0.0|
|   0.0|[10.9273653957852...|[0.54636826978926...|       0.0|
|   0.0|[17.2350134729878...|[0.86175067364939...|       0.0|
|   0.0|[19.4279138082839...|[0.97139569041419...|       0.0|
|   0.0|[19.3499917303618...|[0.96749958651809...|       0.0|
|   0.0|[11.7540320624519...|[0.58770160312259...|       0.0|
|   0.0|[19.3802947606648...|[0.96901473803324...|       0.0|
|   0.0|[19.3499917303618...|[0.96749958651809...|       0.0|
|   0.0|[19.4100785223947...|[0.97050392611973...|       0.0|
|   0.0|[16.9114515818462...|[0.84557257909231...|       0.0|
|   0.0|

In [ ]:
multi_evaluator.evaluate(RF_pred)

0.782051282051282

##NaiveBayes

In [ ]:
from pyspark.ml.classification import NaiveBayes

In [ ]:
NB = NaiveBayes(featuresCol='features', labelCol='Target')

In [ ]:
NB_Model = RF.fit(train_df)

In [ ]:
NB_pred = NB_Model.transform(test_df)

In [ ]:
NB_pred.select('target','rawPrediction','probability','prediction').show()

+------+--------------------+--------------------+----------+
|target|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+----------+
|   0.0|[19.0510579915350...|[0.95255289957675...|       0.0|
|   0.0|[17.4989962388558...|[0.87494981194279...|       0.0|
|   0.0|[16.4095559101595...|[0.82047779550797...|       0.0|
|   0.0|[17.4428405355814...|[0.87214202677907...|       0.0|
|   0.0|[10.9273653957852...|[0.54636826978926...|       0.0|
|   0.0|[17.2350134729878...|[0.86175067364939...|       0.0|
|   0.0|[19.4279138082839...|[0.97139569041419...|       0.0|
|   0.0|[19.3499917303618...|[0.96749958651809...|       0.0|
|   0.0|[11.7540320624519...|[0.58770160312259...|       0.0|
|   0.0|[19.3802947606648...|[0.96901473803324...|       0.0|
|   0.0|[19.3499917303618...|[0.96749958651809...|       0.0|
|   0.0|[19.4100785223947...|[0.97050392611973...|       0.0|
|   0.0|[16.9114515818462...|[0.84557257909231...|       0.0|
|   0.0|

In [ ]:
multi_evaluator.evaluate(NB_pred)

0.782051282051282

##k-Nearest Neighbors (k-NN)

In [ ]:
from pyspark.ml.classification import KNN

ImportError: ignored

##Gradient Boosting

In [ ]:
from pyspark.ml.classification import GBTClassifier

In [ ]:
GB = NaiveBayes(featuresCol='features', labelCol='Target')

In [ ]:
GB_Model = GB.fit(train_df)

In [ ]:
GB_pred = GB_Model.transform(test_df)

In [ ]:
GB_pred.select('target','rawPrediction','probability','prediction').show()

+------+--------------------+--------------------+----------+
|target|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+----------+
|   0.0|[-369.70143258477...|[0.98980940109783...|       0.0|
|   0.0|[-499.37399172930...|[0.98664212299149...|       0.0|
|   0.0|[-654.30500653021...|[0.81176366704901...|       0.0|
|   0.0|[-440.09044061094...|[0.99999881536296...|       0.0|
|   0.0|[-678.28610815209...|[0.99998876422736...|       0.0|
|   0.0|[-618.81683081733...|[0.82930213497668...|       0.0|
|   0.0|[-527.36500059808...|[0.94127500679337...|       0.0|
|   0.0|[-223.22873562558...|[0.19001974407736...|       1.0|
|   0.0|[-784.18710242606...|[0.99939842749418...|       0.0|
|   0.0|[-406.82758735140...|[0.99998608978725...|       0.0|
|   0.0|[-305.94005420966...|[0.99770077152527...|       0.0|
|   0.0|[-350.85567985717...|[0.99996613090899...|       0.0|
|   0.0|[-631.98412004471...|[0.99217391235803...|       0.0|
|   0.0|

In [ ]:
multi_evaluator.evaluate(GB_pred)

0.6452991452991453

##LinearSVC

In [ ]:
from pyspark.ml.classification import LinearSVC

In [ ]:
svc = NaiveBayes(featuresCol='features', labelCol='Target')

In [ ]:
svc_Model = svc.fit(train_df)

In [ ]:
svc_pred = svc_Model.transform(test_df)

In [ ]:
svc_pred.select('target','rawPrediction','probability','prediction').show()

+------+--------------------+--------------------+----------+
|target|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+----------+
|   0.0|[-369.70143258477...|[0.98980940109783...|       0.0|
|   0.0|[-499.37399172930...|[0.98664212299149...|       0.0|
|   0.0|[-654.30500653021...|[0.81176366704901...|       0.0|
|   0.0|[-440.09044061094...|[0.99999881536296...|       0.0|
|   0.0|[-678.28610815209...|[0.99998876422736...|       0.0|
|   0.0|[-618.81683081733...|[0.82930213497668...|       0.0|
|   0.0|[-527.36500059808...|[0.94127500679337...|       0.0|
|   0.0|[-223.22873562558...|[0.19001974407736...|       1.0|
|   0.0|[-784.18710242606...|[0.99939842749418...|       0.0|
|   0.0|[-406.82758735140...|[0.99998608978725...|       0.0|
|   0.0|[-305.94005420966...|[0.99770077152527...|       0.0|
|   0.0|[-350.85567985717...|[0.99996613090899...|       0.0|
|   0.0|[-631.98412004471...|[0.99217391235803...|       0.0|
|   0.0|

In [ ]:
multi_evaluator.evaluate(svc_pred)

0.6452991452991453